In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import logging
import numpy as np
from tqdm import tqdm

In [3]:
from simulator.lob import OrderBookSimulator
from scripts.utils.data_loading import get_data_file_paths

In [4]:
logging.getLogger().setLevel(logging.ERROR)

In [5]:
DATA_ROOT = '../data/onetick/processed_2/AAPL'
IMPACT_PARAM = 10 ** 8  # large values execute against historic orders
DURATION = 30 * 60  # in seconds
SAMPLING_FREQUENCY = 5  # in seconds, dicated by LOB data
NUM_PERIODS = int(DURATION / SAMPLING_FREQUENCY)

In [6]:
simulation_periods = get_data_file_paths(DATA_ROOT)

### Strategy which places a market sell order every PLACEMENT_FREQUENCY periods

In [8]:
calc_midprice = lambda x: (x.BID_PRICE.max() + x.ASK_PRICE.min()) / 2

In [9]:
PLACEMENT_FREQUENCY = 12
ALL_DATA = {}

for sim_data_path in tqdm(simulation_periods):

    # set up LOB simulator
    order_book_file = os.path.join(sim_data_path, 'ob.feather')
    trades_file = os.path.join(sim_data_path, 'trades.feather')
    LOB = OrderBookSimulator(order_book_file, trades_file, IMPACT_PARAM)
    
    # we will save our trade data to these lists
    order_list = []
    midprice_series = []
    
    for period in range(0, NUM_PERIODS):
        
        # advance state
        ob, trds, executed_orders, active_limit_order_levels = LOB.iterate()
        
        # place sell order
        if not (period % PLACEMENT_FREQUENCY):
            LOB.place_market_sell_order(100)
        
        # save data
        midprice_series.append(calc_midprice(ob))
        order_list += executed_orders
    
    ALL_DATA[sim_data_path] = {'price_series': midprice_series, 'executed_orders': order_list}

 41%|████      | 1642/3991 [09:07<13:01,  3.01it/s]

StopIteration: 

In [9]:
shortfall_results = []
for sim_data_path, sim_results in ALL_DATA.items():
    vol_weighted_price_sum = sum([order['volume'] * order['price'] for order in sim_results['executed_orders']])
    total_volume = sum([order['volume'] for order in sim_results['executed_orders']])
    vwap = vol_weighted_price_sum / total_volume
    initial_price = sim_results['price_series'][0]
    shortfall = 100 * (vwap - initial_price) / initial_price
    shortfall_results.append(shortfall)
print("Mean Shortfall: {:.2f}%".format(np.mean(shortfall_results)))
print("StdDev Shortfall: {:.2f}%".format(np.std(shortfall_results)))

Mean Shortfall: 0.02%
StdDev Shortfall: 0.07%
